# Model Setup
Model 1 takes in img outputs moving probability
Model 2 takes in a sequence of 16 moving probs outputs falling probability

In [1]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

import cv2
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from PIL import Image

/home/clarence/.local/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/clarence/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to

GPU limitation

In [11]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

Virtual devices cannot be modified after being initialized


# Building the Models

## Model 1

### Train

In [4]:
data = DataLoader.from_folder(r"./datasets/model1_data")

folder_path = './datasets/model1_data/new-moving'
image_files = [file for file in os.listdir(folder_path)]
augmented_images = []  # List to store augmented images
data_augmentation = tf.keras.Sequential([
  layers.RandomRotation(0.26),
])
cnt = 0
# Apply data augmentation to each image and store the augmented images
for image_file in image_files:
    if cnt == 1:
        break
    image_path = os.path.join(folder_path, image_file)
    
    image = tf.io.read_file(image_path)
    image = tf.io.decode_jpeg(image, channels=3)  
    image = tf.expand_dims(image, axis=0)
    # Apply data augmentation
    augmented_image = data_augmentation(image)
    augmented_images.append(augmented_image)
    cnt+=1

# Specify the path of the folder where you want to save the augmented images
save_folder = './datasets/model1_data/augmented'

# Iterate through augmented_images and save them to the specified folder
for i, augmented_image in enumerate(augmented_images):
    # Generate a unique file name for the saved image
    image_name = f'augmented_image_{i+210}.jpg'
    
    # Save the augmented image to the specified folder
    save_path = os.path.join(save_folder, image_name)
    tf.keras.preprocessing.image.save_img(save_path, augmented_image[0])

train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 5211, num_label: 5, labels: augmented, moving, new-moving, new-still, still.


: 

In [4]:
model = image_classifier.create(train_data, use_augmentation=True)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 3,415,586
Trainable params: 2,562
Non-trainable params: 3,413,024
_________________________________________________________________
None
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


Epoch 1/5


2023-06-01 13:46:46.920180: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


82/82 [==============================] - 4s 28ms/step - loss: 0.4467 - accuracy: 0.8304
Epoch 2/5
82/82 [==============================] - 2s 28ms/step - loss: 0.3599 - accuracy: 0.9017
Epoch 3/5
82/82 [==============================] - 2s 29ms/step - loss: 0.3556 - accuracy: 0.8921
Epoch 4/5
82/82 [==============================] - 2s 28ms/step - loss: 0.3352 - accuracy: 0.9181
Epoch 5/5
82/82 [==============================] - 2s 28ms/step - loss: 0.3385 - accuracy: 0.9135


### Evaluate & Export

In [5]:
loss, accuracy = model.evaluate(test_data)
# model.summary()

10/10 [==============================] - 1s 45ms/step - loss: 0.2942 - accuracy: 0.9521


In [6]:
model.export(export_dir='.')

2023-05-31 14:30:43.652768: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmp62fgqcij/assets


INFO:tensorflow:Assets written to: /tmp/tmp62fgqcij/assets
/home/clarence/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-31 14:30:47.189791: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-05-31 14:30:47.189821: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 3
INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmp3ugu9wk9/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmp3ugu9wk9/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


## Model 2

### Train

In [7]:
default_data = np.loadtxt('./datasets/model2_data/default.txt')
falling_data = np.loadtxt('./datasets/model2_data/falling.txt')

In [8]:
inputs = np.concatenate((default_data, falling_data))
outputs = np.concatenate((np.zeros(len(default_data)), np.ones(len(falling_data))))

# Convert inputs and outputs to TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))
dataset = dataset.shuffle(len(inputs)).batch(128)

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(16,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [10]:
model.fit(dataset, epochs=6,callbacks=[tensorboard_callback])

Epoch 1/6
5/5 [==============================] - 0s 2ms/step - loss: 0.7032 - accuracy: 0.6275
Epoch 2/6
5/5 [==============================] - 0s 2ms/step - loss: 0.6862 - accuracy: 0.6438
Epoch 3/6
5/5 [==============================] - 0s 1ms/step - loss: 0.6705 - accuracy: 0.6908
Epoch 4/6
5/5 [==============================] - 0s 1ms/step - loss: 0.6555 - accuracy: 0.7414
Epoch 5/6
5/5 [==============================] - 0s 1ms/step - loss: 0.6409 - accuracy: 0.7703
Epoch 6/6
5/5 [==============================] - 0s 2ms/step - loss: 0.6263 - accuracy: 0.8083


### Evaluate & Export

In [11]:
# loss, acc = model.evaluate(X_test, y_test)
# model.summary()
model.save("model2")

INFO:tensorflow:Assets written to: model2/assets


INFO:tensorflow:Assets written to: model2/assets


# Predict

## Reformat Input

In [12]:
def resize_video(input_path, output_path, width, height):
  # Open the video file
  video = cv2.VideoCapture(input_path)

  # Get the original video's width and height
  original_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  original_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  
  # Create a VideoWriter object to save the resized video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for the output video
  fps = video.get(cv2.CAP_PROP_FPS)
  writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  
  while True:
    # Read a frame from the original video
    ret, frame = video.read()
    if not ret: break
    # Resize the frame to the desired width and height
    resized_frame = cv2.resize(frame, (width, height))
    # Write the resized frame to the output video file
    writer.write(resized_frame)

# Release the video capture and writer objects
  video.release()
  writer.release()

In [50]:
# Resize vid
video_path = "s-walk2.mp4"
output_path = "datasets/model2_vids/resized_c_run1.mp4"
target_width = 224
target_height = 224

# resize_video(video_path, output_path, target_width, target_height)

# Load resized vid
cap = cv2.VideoCapture(output_path)
# frame_rate = 30
# cap.set(cv2.CAP_PROP_FPS, frame_rate) DOESN'T WORK
print(cap.get(cv2.CAP_PROP_FPS))

30.0


## Pass into model 1

In [51]:
# Initialize list to store the frame classifications
frame_classifications = []
moving_prob = []
# Loop through the frames of the video (need to change to 30 fps)
while True:
    ret, frame = cap.read() 
    #just need to figure out if this is 30 fps

    if not ret: # Break the loop if the video has ended
        break

    ''' Formulate Input Data (frame_rgb) '''
    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make frame input data and ensure its type matches the model
    frame_rgb = np.expand_dims(frame_rgb, axis=0)

    ''' Classify the Frame '''
    interpreter = tf.lite.Interpreter(model_path="model.tflite")
    interpreter.allocate_tensors()

    # get_output_details() and get_input_details() return list of dictionaries of tensor details
    # keys: name, index, shape, shape_signature, dtype, quantization, ...
    # len(input) = len(output) = 1, so access the first element
    output = interpreter.get_output_details()
    input = interpreter.get_input_details()
    output_index = output[0]['index']
    input_index = input[0]['index']

    # set input -> invoke -> access output
    interpreter.set_tensor(input_index, frame_rgb)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_index)
    # If the output_data shape is (batch_size, num_classes), select the first frame
    output_data = output_data[0]

    # Convert each entry into probability
    output_probs = tf.nn.softmax(output_data.astype(float))

    # Find the index of the highest probability
    predicted_index = np.argmax(output_data)

    # Assuming you have a list of class labels corresponding to the model's output classes
    class_labels = ["Moving", "Still"]

    # Get the predicted class label
    predicted_class = class_labels[predicted_index]
    
    # Print the predicted class label
    # print("Predicted Class:", predicted_class)
    frame_classifications.append((predicted_class, max(output_probs.numpy())))
    
    prob = np.around(max(output_probs.numpy()), decimals = 2)
    if predicted_class == "Still":
        
        moving_prob.append(np.subtract(1, prob))
    else:
        moving_prob.append(prob)

## Prepare data for model 2

In [52]:
moving_probs_trimmed = moving_prob[:-(len(moving_prob)%16)] if len(moving_prob) % 16 != 0 else moving_prob
model2_in = np.array(moving_probs_trimmed).reshape((len(moving_prob)//16, 16))
# print(model2_in)

## Pass into Model 2 and get final prediction

In [53]:
model2 = tf.keras.models.load_model("model2")

In [54]:
vid_preds = model2.predict(model2_in)

In [55]:
bools = vid_preds.reshape((1, len(vid_preds))) > 0.5
print(bools)

[[ True  True False False False False False False False False False]]
